Different models, best val loss = 0.17

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import layers
df = pd.read_csv("drive/MyDrive/Engineer's Project/test.csv")

In [ ]:
df.pop('Date')
df.head()

,Opening,High,Low,Closing,Momentum,Range,ohlc,usa_cpi,pol_cpi,usa_inter,pol_inter
0,2.879833,2.8879,2.8701,2.879821,0.000011,0.0178,2.879414,1.143161,2.7,2.76,5.82
1,2.884751,2.8941,2.8741,2.884744,0.000007,0.0200,2.884424,1.143161,2.7,2.76,5.82
2,2.922487,2.9499,2.8787,2.922526,-0.000040,0.0712,2.918403,1.143161,2.7,2.76,5.82
3,2.924881,2.9462,2.9071,2.924866,0.000015,0.0391,2.925762,1.143161,2.7,2.76,5.82
4,2.889930,2.9105,2.8762,2.889919,0.000011,0.0343,2.891637,1.143161,2.7,2.76,5.82


In [ ]:
float_data = np.array(df).astype('float32')

In [ ]:
mean = float_data[:2500].mean(axis=0)
float_data -= mean
std = float_data[:2500].std(axis=0)
float_data /= std

In [ ]:
lookback = 30
step = 1
delay =1
min_index  = 0
max_index = 2500
#max_index  = 3541
batch_size = 128

In [ ]:
def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=1):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while 1:
    if shuffle:
      rows = np.random.randint(
        min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][1]
    yield samples, targets

In [ ]:
train_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=2500,
#max_index=3541,
shuffle=True,
step=step,
batch_size=batch_size)

In [ ]:
val_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=2501,
max_index=3000,
step=step,
batch_size=batch_size)

In [ ]:
test_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=3001,
max_index=3541,
step=step,
batch_size=batch_size)

In [ ]:
val_steps = (3000 - 2501 - lookback)
test_steps = (len(float_data) - 3001 - lookback)

In [ ]:
model5 = tf.keras.models.Sequential()
model5.add(layers.LSTM(64,
  #dropout=0.1,
  #recurrent_dropout=0.1,
  return_sequences=True,
  input_shape=(None, float_data.shape[-1])))
model5.add(layers.LSTM(64,
  #dropout=0.1,
  #recurrent_dropout=0.1,
  input_shape=(None, float_data.shape[-1])))
model5.add(layers.Dense(64))
model5.add(layers.Dense(1))
model5.compile(optimizer= tf.keras.optimizers.RMSprop(), loss='mae')
history5 = model5.fit(train_gen,
  steps_per_epoch=200,
  epochs=12,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/12
200/200 [==============================] - 31s 136ms/step - loss: 0.1145 - val_loss: 0.3042
Epoch 2/12
200/200 [==============================] - 26s 131ms/step - loss: 0.0836 - val_loss: 0.2918
Epoch 3/12
200/200 [==============================] - 27s 136ms/step - loss: 0.0761 - val_loss: 0.2542
Epoch 4/12
200/200 [==============================] - 27s 134ms/step - loss: 0.0711 - val_loss: 0.2523
Epoch 5/12
200/200 [==============================] - 26s 132ms/step - loss: 0.0675 - val_loss: 0.2066
Epoch 6/12
200/200 [==============================] - 27s 134ms/step - loss: 0.0648 - val_loss: 0.2007
Epoch 7/12
200/200 [==============================] - 27s 135ms/step - loss: 0.0612 - val_loss: 0.2845
Epoch 8/12
200/200 [==============================] - 27s 134ms/step - loss: 0.0589 - val_loss: 0.2339
Epoch 9/12
200/200 [==============================] - 27s 134ms/step - loss: 0.0564 - val_loss: 0.2005
Epoch 10/12
200/200 [==============================] - 27s 133ms/step - l

WOW!

In [ ]:
eval5 = model5.evaluate(test_gen, steps = test_steps)
eval5 *std[6]

511/511 [==============================] - 15s 29ms/step - loss: 0.1766


0.06594478231906908

ADAM instead of RMSprop

In [ ]:
model6 = tf.keras.models.Sequential()
model6.add(layers.GRU(32,
  #dropout=0.1,
  #recurrent_dropout=0.1,
  return_sequences=True,
  input_shape=(None, float_data.shape[-1])))
model6.add(layers.GRU(64,
  #dropout=0.1,
  #recurrent_dropout=0.1,
  return_sequences=True,
  input_shape=(None, float_data.shape[-1])))
model6.add(layers.LSTM(128,
  #dropout=0.1,
  #recurrent_dropout=0.1,
  input_shape=(None, float_data.shape[-1])))
model6.add(layers.Dense(64))
model6.add(layers.Dense(1))
model6.compile(optimizer= tf.keras.optimizers.RMSprop(), loss='mae')
history6 = model6.fit(train_gen,
  steps_per_epoch=200,
  epochs=12,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/12
200/200 [==============================] - 56s 255ms/step - loss: 0.1356 - val_loss: 0.2330
Epoch 2/12
200/200 [==============================] - 50s 249ms/step - loss: 0.0944 - val_loss: 0.4246
Epoch 3/12
200/200 [==============================] - 50s 250ms/step - loss: 0.0849 - val_loss: 0.2883
Epoch 4/12
200/200 [==============================] - 50s 250ms/step - loss: 0.0795 - val_loss: 0.3127
Epoch 5/12
200/200 [==============================] - 49s 246ms/step - loss: 0.0750 - val_loss: 0.3190
Epoch 6/12
200/200 [==============================] - 49s 244ms/step - loss: 0.0721 - val_loss: 0.3183
Epoch 7/12
200/200 [==============================] - 48s 243ms/step - loss: 0.0693 - val_loss: 0.2281
Epoch 8/12
200/200 [==============================] - 48s 243ms/step - loss: 0.0656 - val_loss: 0.2774
Epoch 9/12
200/200 [==============================] - 48s 242ms/step - loss: 0.0622 - val_loss: 0.2857
Epoch 10/12
200/200 [==============================] - 49s 245ms/step - l

In [ ]:
eval6 = model6.evaluate(test_gen, steps = test_steps)
eval6 *std[6]

511/511 [==============================] - 27s 52ms/step - loss: 0.2229


0.08321104263750012

In [ ]:
del model6

In [ ]:
eval7 = model7.evaluate(test_gen, steps = test_steps)
eval7 *std[6]

511/511 [==============================] - 20s 38ms/step - loss: 0.3883


0.14495999005955973

In [ ]:
model7 = tf.keras.models.Sequential()
model7.add(layers.GRU(32,
  dropout=0.1,
  recurrent_dropout=0.1,
  return_sequences=True,
  input_shape=(None, float_data.shape[-1])))
model7.add(layers.GRU(64,
  dropout=0.1,
  recurrent_dropout=0.1,
  return_sequences=True,
  input_shape=(None, float_data.shape[-1])))
model7.add(layers.LSTM(128,
  dropout=0.1,
  recurrent_dropout=0.1,
  input_shape=(None, float_data.shape[-1])))
model7.add(layers.Dense(64))
model7.add(layers.Dense(1))
model7.compile(optimizer= tf.keras.optimizers.RMSprop(), loss='mae')
history7 = model7.fit(train_gen,
  steps_per_epoch=200,
  epochs=12,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/12
200/200 [==============================] - 72s 315ms/step - loss: 0.1498 - val_loss: 0.1594
Epoch 2/12
200/200 [==============================] - 61s 307ms/step - loss: 0.1124 - val_loss: 0.2611
Epoch 3/12
200/200 [==============================] - 61s 308ms/step - loss: 0.1014 - val_loss: 0.2787
Epoch 4/12
200/200 [==============================] - 60s 302ms/step - loss: 0.0972 - val_loss: 0.2566
Epoch 5/12
200/200 [==============================] - 61s 304ms/step - loss: 0.0921 - val_loss: 0.2634
Epoch 6/12
200/200 [==============================] - 62s 309ms/step - loss: 0.0889 - val_loss: 0.2882
Epoch 7/12
200/200 [==============================] - 61s 303ms/step - loss: 0.0851 - val_loss: 0.3272
Epoch 8/12
200/200 [==============================] - 60s 303ms/step - loss: 0.0825 - val_loss: 0.2402
Epoch 9/12
200/200 [==============================] - 61s 305ms/step - loss: 0.0785 - val_loss: 0.3131
Epoch 10/12
200/200 [==============================] - 61s 304ms/step - l

In [ ]:
model1 = tf.keras.models.Sequential()
model1.add(layers.GRU(20,
  #dropout=0.1,
  #recurrent_dropout=0.1,
  return_sequences=True,
  input_shape=(None, float_data.shape[-1])))
model1.add(layers.LSTM(256,
  dropout=0.1,
  #recurrent_dropout=0.1,
  input_shape=(None, float_data.shape[-1])))
model1.add(layers.Dense(64))
model1.add(layers.Dense(1))
model1.compile(optimizer= tf.keras.optimizers.RMSprop(), loss='mae')
history1 = model1.fit(train_gen,
  steps_per_epoch=200,
  epochs=20,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/20
200/200 [==============================] - 101s 485ms/step - loss: 0.1353 - val_loss: 0.2076
Epoch 2/20
200/200 [==============================] - 93s 465ms/step - loss: 0.0990 - val_loss: 0.2110
Epoch 3/20
200/200 [==============================] - 91s 458ms/step - loss: 0.0898 - val_loss: 0.1679
Epoch 4/20
200/200 [==============================] - 92s 463ms/step - loss: 0.0844 - val_loss: 0.3248
Epoch 5/20
200/200 [==============================] - 94s 472ms/step - loss: 0.0808 - val_loss: 0.2022
Epoch 6/20
200/200 [==============================] - 93s 465ms/step - loss: 0.0770 - val_loss: 0.1973
Epoch 7/20
200/200 [==============================] - 94s 469ms/step - loss: 0.0752 - val_loss: 0.2188
Epoch 8/20
200/200 [==============================] - 93s 465ms/step - loss: 0.0721 - val_loss: 0.1862
Epoch 9/20
200/200 [==============================] - 95s 474ms/step - loss: 0.0690 - val_loss: 0.2101
Epoch 10/20
200/200 [==============================] - 94s 470ms/step - 

In [ ]:
eval1 = model1.evaluate(test_gen, steps = test_steps)
eval1 *std[6]

511/511 [==============================] - 47s 91ms/step - loss: 0.2918


0.10892725858839825

In [ ]:
del model1

NameError: ignored